---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [0]:
from google.colab import drive
drive.mount('/data')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /data


In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [0]:
!pip install xlrd

    100% |████████████████████████████████| 112kB 2.7MB/s 


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [0]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [0]:
def get_list_of_university_towns():
  '''Returns a DataFrame of towns and the states they are in from the 
  university_towns.txt list. The format of the DataFrame should be:
  DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
  columns=["State", "RegionName"]  )

  The following cleaning needs to be done:

  1. For "State", removing characters from "[" to the end.
  2. For "RegionName", when applicable, removing every character from " (" to the end.
  3. Depending on how you read the data, you may need to remove newline character '\n'. '''
  df = pd.read_table('/data/My Drive/DS1/data/university_towns.txt', header=None)
  df.columns = ['StateRegion']
  df['StateFlag'] = df['StateRegion'].str.contains(pat=r'\[edit\]')
  df_states = df[df['StateFlag'] == True]
  States = df_states['StateRegion'].str.replace(pat=r'\[edit\]', repl='').unique()
  StateFlags = df['StateFlag'].values
  j = 0
  StateMatchRegion = [States[j]]
  for i in range(1, len(StateFlags)):
    if StateFlags[i] == True:
      j += 1
      StateMatchRegion.append(States[j])
    else:
      j += 0
      StateMatchRegion.append(States[j])
  df['StateMatchRegion'] = StateMatchRegion
  df['RegionName'] = df['StateRegion'].str.replace(pat=r'\s\(.*', repl='')
  ans = df[df['StateFlag'] == False]
  ans = ans.drop(['StateRegion', 'StateFlag'], axis=1)
  ans.columns = ['State', 'RegionName']
  mapping_df = pd.DataFrame({
      "State": list(states.values()),
      "StateAcronyms": list(states.keys())
  })
  ans = pd.merge(ans, mapping_df, how='left')
  return ans

get_list_of_university_towns()

,State,RegionName,StateAcronyms
0,Alabama,Auburn,AL
1,Alabama,Florence,AL
2,Alabama,Jacksonville,AL
3,Alabama,Livingston,AL
4,Alabama,Montevallo,AL
5,Alabama,Troy,AL
6,Alabama,Tuscaloosa,AL
7,Alabama,Tuskegee,AL
8,Alaska,Fairbanks,AK
9,Arizona,Flagstaff,AZ


In [0]:
def get_recession_start():
  """
  Returns the year and quarter of the recession start time as a 
  string value in a format such as 2005q3
  """
  df = pd.read_excel('/data/My Drive/DS1/data/gdplev.xls', skiprows=220, header=None)
  df = df.iloc[:, [4, 6]]
  df.columns = ['Quarterly', 'GDP in billions of chained 2009 dollars']
  df_shift = df.shift(1)
  df_concat = pd.concat([df, df_shift['GDP in billions of chained 2009 dollars']], axis = 1)
  df_concat['Quarterly Difference'] = df_concat.iloc[:, 1] - df_concat.iloc[:, 2]
  is_recession_start = [False, False]
  quarterly_diff = df_concat['Quarterly Difference'].values
  for i in range(2, len(quarterly_diff)):
    if quarterly_diff[i] < 0 and quarterly_diff[i - 1] < 0:
      is_recession_start.append(True)
    else:
      is_recession_start.append(False)
  df_concat['is_recession_start'] = is_recession_start
  ans = df_concat[df_concat['is_recession_start'] == True]['Quarterly'].values[0]
  return ans

get_recession_start()


'2008q4'

In [0]:
def get_recession_end():
  '''Returns the year and quarter of the recession end time as a 
  string value in a format such as 2005q3'''
  df = pd.read_excel('/data/My Drive/DS1/data/gdplev.xls', skiprows=220, header=None)
  df = df.iloc[:, [4, 6]]
  df.columns = ['Quarterly', 'GDP in billions of chained 2009 dollars']
  df_shift = df.shift(1)
  df_concat = pd.concat([df, df_shift['GDP in billions of chained 2009 dollars']], axis = 1)
  df_concat['Quarterly Difference'] = df_concat.iloc[:, 1] - df_concat.iloc[:, 2]
  is_recession_start = [False, False]
  quarterly_diff = df_concat['Quarterly Difference'].values
  for i in range(2, len(quarterly_diff)):
    if quarterly_diff[i] < 0 and quarterly_diff[i - 1] < 0:
      is_recession_start.append(True)
    else:
      is_recession_start.append(False)
  df_concat['is_recession_start'] = is_recession_start
  recession_start_idx = df_concat[df_concat['is_recession_start'] == True]['Quarterly'].values[0]
  df_concat = df_concat.set_index('Quarterly')
  df_after_recession = df_concat.loc[recession_start_idx:, :].drop('is_recession_start', axis = 1)
  is_recession_end = [False, False]
  quarterly_diff = df_after_recession['Quarterly Difference'].values
  for i in range(2, len(quarterly_diff)):
    if quarterly_diff[i] > 0 and quarterly_diff[i - 1] > 0:
      is_recession_end.append(True)
    else:
      is_recession_end.append(False)
  ans = df_after_recession[is_recession_end].index[1]
  return ans

get_recession_end()

'2010q1'

In [0]:
def get_recession_bottom():
  '''Returns the year and quarter of the recession bottom time as a 
  string value in a format such as 2005q3'''
  df = pd.read_excel('/data/My Drive/DS1/data/gdplev.xls', skiprows=220, header=None)
  df = df.iloc[:, [4, 6]]
  df.columns = ['Quarterly', 'GDP in billions of chained 2009 dollars']
  df = df.set_index('Quarterly')
  recession_starts = get_recession_start()
  recession_ends = get_recession_end()
  df_recession = df.loc[recession_starts:recession_ends, :]
  ans = df_recession.idxmin().values[0]
  return ans

get_recession_bottom()

'2009q2'

In [0]:
def convert_housing_data_to_quarters():
  '''Converts the housing data to quarters and returns it as mean 
  values in a dataframe. This dataframe should be a dataframe with
  columns for 2000q1 through 2016q3, and should have a multi-index
  in the shape of ["State","RegionName"].

  Note: Quarters are defined in the assignment description, they are
  not arbitrary three month periods.

  The resulting dataframe should have 67 columns, and 10,730 rows.
  '''
  df = pd.read_csv('/data/My Drive/DS1/data/City_Zhvi_AllHomes.csv')
  df_state_region = df.loc[:, ['State', 'RegionName']]
  df_housing_price = df.loc[:, '2000-01':]
  months = df_housing_price.columns
  quarter_means = []
  for m in range(0, len(months), 3):
    df_subset = df_housing_price.iloc[:, m:m+2]
    quarter_mean = df_subset.mean(axis=1)
    quarter_means.append(quarter_mean)
  Quarters = []
  for y in range(2000, 2017):
    for q in ['q1', 'q2', 'q3', 'q4']:
      Quarters.append(str(y) + q)
  Quarters.pop()
  df_concat = quarter_means[0]
  for i in range(1, len(quarter_means)):
    df_concat = pd.concat([df_concat, quarter_means[i]], axis = 1)
  df_concat.columns = Quarters
  df_concat = pd.concat([df_state_region, df_concat], axis = 1)
  ans = df_concat.set_index(['State', 'RegionName'])
  
  return ans

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,512800.0,522300.0,527300.0,531500.0,538400.0,554300.0,571150.0,580300.0,592350.0,587200.0
CA,Los Angeles,205700.0,213400.0,220050.0,225200.0,231800.0,238200.0,243950.0,251600.0,260350.0,270600.0,...,496150.0,507300.0,517450.0,527000.0,536750.0,545600.0,556200.0,564200.0,575900.0,584050.0
IL,Chicago,137550.0,142800.0,147300.0,151200.0,156300.0,160850.0,165850.0,169600.0,174750.0,177700.0,...,192500.0,194750.0,200900.0,200850.0,205250.0,207900.0,208400.0,206000.0,207750.0,212000.0
PA,Philadelphia,52900.0,53550.0,53950.0,54650.0,55250.0,55450.0,56100.0,57250.0,58850.0,60500.0,...,113550.0,115200.0,115500.0,116100.0,117450.0,120700.0,122500.0,122550.0,126700.0,128700.0
AZ,Phoenix,111350.0,114000.0,115750.0,117300.0,119050.0,121350.0,122550.0,123950.0,126200.0,128150.0,...,164150.0,164950.0,167800.0,171250.0,173500.0,178100.0,183100.0,187300.0,190750.0,195200.0
NV,Las Vegas,132150.0,134250.0,135100.0,136650.0,139050.0,141550.0,142950.0,145550.0,149000.0,150700.0,...,169350.0,173000.0,175000.0,177250.0,180450.0,186250.0,189950.0,194200.0,196700.0,199950.0
CA,San Diego,221050.0,232300.0,243650.0,254250.0,265150.0,275150.0,283250.0,290500.0,299700.0,310250.0,...,478750.0,487450.0,495500.0,502050.0,510500.0,518750.0,524850.0,528500.0,535400.0,539750.0
TX,Dallas,84800.0,83700.0,84700.0,86950.0,89750.0,89400.0,89000.0,90450.0,92350.0,93600.0,...,106350.0,108450.0,111150.0,112950.0,119750.0,127500.0,133600.0,139650.0,143900.0,149300.0
CA,San Jose,369050.0,401400.0,428200.0,451800.0,469650.0,471650.0,458550.0,447900.0,439900.0,454350.0,...,676400.0,693700.0,712350.0,727900.0,752200.0,774200.0,787650.0,799450.0,818350.0,822200.0


In [0]:
def run_ttest():
  '''First creates new data showing the decline or growth of housing prices
  between the recession start and the recession bottom. Then runs a ttest
  comparing the university town values to the non-university towns values, 
  return whether the alternative hypothesis (that the two groups are the same)
  is true or not as well as the p-value of the confidence. 

  Return the tuple (different, p, better) where different=True if the t-test is
  True at a p<0.01 (we reject the null hypothesis), or different=False if 
  otherwise (we cannot reject the null hypothesis). The variable p should
  be equal to the exact p value returned from scipy.stats.ttest_ind(). The
  value for better should be either "university town" or "non-university town"
  depending on which has a lower mean price ratio (which is equivilent to a
  reduced market loss).'''
  housing_data_quarters = convert_housing_data_to_quarters()
  recession_start = get_recession_start()
  recession_bottom = get_recession_bottom()
  quarters = housing_data_quarters.columns
  recession_start_idx = list(quarters).index(recession_start)
  before_recession = quarters[:recession_start_idx][-1]
  new_data = housing_data_quarters.loc[:, [before_recession, recession_bottom]].reset_index()
  university_towns = get_list_of_university_towns().drop('State', axis=1)
  new_data = pd.merge(new_data, university_towns, how='left', left_on = ['State', 'RegionName'], right_on = ['StateAcronyms', 'RegionName'])
  new_data = new_data.assign(is_university_town = [True if isinstance(x, str) else False for x in new_data['StateAcronyms'].values])
  new_data = new_data.drop('StateAcronyms', axis=1)
  new_data = new_data.assign(price_ratio = new_data[before_recession] / new_data[recession_bottom])
  university_towns = new_data[new_data['is_university_town']]['price_ratio'].values
  non_university_towns = new_data[new_data['is_university_town'] == False]['price_ratio'].values
  ttest_res = ttest_ind(university_towns, non_university_towns, nan_policy='omit')
  reject_null_hypothesis = None
  lower_mean_price_ratio = None
  if ttest_res[1] < 0.01:
    reject_null_hypothesis = True
  else:
    reject_null_hypothesis = False
  if ttest_res[0] < 0:
    lower_mean_price_ratio = "university town"
  else:
    lower_mean_price_ratio = "non-university town"
  return reject_null_hypothesis, ttest_res[1], lower_mean_price_ratio

run_ttest()


(True, 0.005366703729629153, 'university town')